In [73]:
import numpy as np
import scipy.optimize as sp
import itertools
import os

### Define potentials to be optimized against

In [18]:
# Assume all constants are 1 for simplicity
epsilon = 1
lj_sigma = 1
r_e = 1
m_sigma = 1
d_e = 1

def lj(r):
    '''Calculate lennard-jones potential between two particles a distance r apart'''
    u_lj = 4*epsilon*((lj_sigma/r)**12-(lj_sigma/r)**6)
    return u_lj

def Morse(r):
    '''Calculate morse potential between two particles a distance r apart'''
    u_m = d_e*(1-np.exp(-(r-r_e)/m_sigma))**2
    return u_m

In [40]:
class vec3d(np.ndarray):
    """ A 3D cartesian vector."""
    def __new__(cls, lst):
        """Create a new instance of the vec3d by calling the ndarray __new__."""
        ar = np.array(lst, dtype=float)
        x = np.ndarray.__new__(cls, shape=(3,), dtype=float, buffer=ar)
        return x

    def length(self):
        """Returns the length of the vector."""
        return np.sqrt(np.sum(self*self))
    
    def coords(self):
        '''Returns coordinates as a 1x3 array of x,y,z coords'''
        xyz = np.array((self[0],self[1],self[2]))
        return xyz

In [82]:
class System():
    '''Interface to calculate and optimize potentials'''
    def __init__(self,n_particles,potential_func):
        '''
        Place n particles in random positions in a 10x10x10 box, cornered on the origin
        Particle positions are stored in a list of vec3d objects
        '''
        self.n_particles = n_particles
        self.potential_func = potential_func

        self.particles = list()
        for position in np.random.rand(self.n_particles,3)*10:
            self.particles.append(vec3d(position))


    def potential_energy(self):
        '''
        Calculate the potential energy of the current system state by summing 
        the potentials between all pairs of particles
        '''
        total_potential = 0

        for i,j in itertools.combinations(range(self.n_particles), 2): #using itertools to generate i,j for all possible pairs 
            distance = self.particles[i]-self.particles[j]
            distance = distance.length()
            total_potential += self.potential_func(distance)
        return total_potential

    def flatten(self):
        '''Output flattened array of particle coordinates for minimize function'''
        flat = np.array(())
        for p in self.particles:
            flat = np.append(flat,p.coords())
        return flat

    def unflatten(self,flat):
        '''Take flattened array from scipy input and load it back into the system for calculation'''
        unflatten = list()
        for i in range(0,self.n_particles):
            unflatten.append(vec3d(flat[i*3:i*3+3]))
        return unflatten

    def optimize_geom(self):
        '''
        Optimize positions of particles stored in the system class relative to
        the stored potential function

        '''
        def opt_function(coords_flat):
            '''
            Input: flat array of the coordinates of particles in the system
            
            Converts flattened array back to vec3d objects then calculates the
            energy using the potential function defined upon system creation
            
            Output: potential energy of the system at the given geometry
            '''
            self.particles = self.unflatten(coords_flat)
            energy = self.potential_energy()
            return energy
        
        x0 = self.flatten()
        result = sp.minimize(opt_function, x0)
        
        opt_energy = result.fun
        self.particles = self.unflatten(result.x)

        # Create output xyz file for the system
        
        return opt_energy,self.particles
    
    # Define function to save to a file
    def save_to_xyz(self,filename):
        '''
        Input: a string 'filename' to be saved as filename.xyz
        Save the result of the optimization to an xyz file
        '''
        with open(filename+'xyz','w') as f:
            f.write(f'{self.n_particles}\n')
            f.write(f'Optmized geometry of {self.n_particles} particles giving min_potential = {self.potential_energy()}\n')
            for particle in self.particles:
                x, y, z = particle.coords()
                f.write(f'C {x}  {y}  {z}\n')

In [74]:
min_potential = np.inf

for i in range(0,20):
    sys = System(4,lj)
    res = sys.optimize_geom()
    if res[0] < min_potential:
        min_potential = res[0]
        min_positions = res[1]
    